In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import requests
import time
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import json

In [450]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [451]:
specializations = {
    "Менеджер": {
        "primary": ["Управление проектом", "Agile", "Scrum", "Kanban", "Jira"],
        "secondary": ["Управление персоналом", "Бизнес-процесс", "Confluence", "MS Project", "PMBoK"],
        "tertiary": ["Тайм-менеджмент", "Проведение презентации", "Google Docs", "Notion", "Разработка концепции"]
    },
    "Аналитик данных": {
        "primary": ["SQL", "Python", "Анализ данных", "Power BI", "Excel"],
        "secondary": ["База данных", "PostgreSQL", "ETL", "Tableau", "BI"],
        "tertiary": ["NumPy", "A/B тест", "Математический анализ", "Pandas", "Scikit-learn"]
    },
    "Специалист по 1С": {
        "primary": ["1С", "1С: Предприятие", "1С: Бухгалтерия", "Консультирование пользователей"],
        "secondary": ["1С: Зарплата и управление персоналом", "1С: Документооборот", "Анализ бизнес-процесса"],
        "tertiary": ["Документооборот", "Техническая поддержка"]
    },
    "Тестировщик": {
        "primary": ["SQL", "API", "REST", "Ручное тестирование", "Функциональное тестирование"],
        "secondary": ["Jira", "Postman", "Тестирование мобильного приложения", "Автоматизация тестирования", "Selenium"],
        "tertiary": ["HTTP", "JSON", "Тестирование API", "Интеграционное тестирование", "Регрессионное тестирование"]
    },
    "Специалист по Data Science": {
        "primary": ["Python", "Machine learning", "SQL", "PyTorch", "NLP"],
        "secondary": ["NumPy", "TensorFlow", "Scikit-learn", "Pandas", "CatBoost"],
        "tertiary": ["Big Data", "Computer vision", "XGBoost", "Deep learning", "OpenCV"]
    },
    "Маркетолог": {
        "primary": ["СustDev", "SEO", "CJM", "Яндекс Директ", "Реклама"],
        "secondary": ["SMM", "Анализ рынка", "Инструменты маркетинга и рекламы Google", "Копирайтинг", "Power BI"],
        "tertiary": ["Маркетинговая стратегия", "E-commerce", "Email-маркетинг", "MyTarget", "B2B-marketing"]
    },
    "Фронтенд-разработчик": {
        "primary": ["JavaScript", "TypeScript", "React", "CSS", "HTML"],
        "secondary": ["Vue.js", "API", "Webpack", "REST", "Redux"],
        "tertiary": ["Sass", "Docker", "JQuery", "Bootstrap", "Node.js"]
    },
    "Системный аналитик": {
        "primary": ["SQL", "REST", "UML", "BPMN", "Jira"],
        "secondary": ["Confluence", "Agile", "Scrum", "Kanban", "ER-model"],
        "tertiary": ["Visio", "JSON", "XML", "SoapUI", "Flowchart"]
    },
    "Специалист по БАС": {
        "primary": ["Python", "SITL", "Ardupilot", "Робототехника", "Embedded", "Arduino", "OrangePi", "RaspberryPi"],
        "secondary": ["QGroundControl", "ROS", "SolidWorks", "Mission Planner", "3D-моделирование"],
        "tertiary": ["Авиастроение", "Pixhawk", "КОМПАС-3D", "3D-печать", "C/C++"]
    },
    "DevOps инженер": {
        "primary": ["Linux", "CI/CD", "Docker", "Kubernetes", "Ansible"],
        "secondary": ["Grafana", "Prometheus", "Git", "Jenkins", "Terraform"],
        "tertiary": ["Redis", "MongoDB", "Hadoop", "Airflow", "Kafka"]
    },
    "Дизайнер": {
        "primary": ["UX", "Figma", "UI", "Adobe Photoshop", "Adobe Illustrator"],
        "secondary": ["Web design", "Tilda", "Adobe XD", "Sketch", "Axure RP"],
        "tertiary": ["Miro", "Landing page", "Дизайн полиграфии", "Adobe After Effects", "Брендинг"]
    },
    "Инженер данных": {
        "primary": ["SQL", "Python", "ETL", "PostgreSQL", "Airflow"],
        "secondary": ["Spark", "Kafka", "ClickHouse", "MS SQL", "Oracle"],
        "tertiary": ["Greenplum", "MongoDB", "Redis", "Superset", "DWH"]
    },
    "Фулстек-разработчик": {
        "primary": ["JavaScript", "Python", "SQL", "React", "PostgreSQL"],
        "secondary": ["Node.js", "REST", "MongoDB", "Docker", "HTML"],
        "tertiary": ["GraphQL", "SOAP", "Bootstrap", "MySQL", "FastAPI"]
    },
    "Бэкенд-разработчик": {
        "primary": ["Python", "Docker", "REST", "PostgreSQL", "SQL"],
        "secondary": ["RabbitMQ", "Kafka", "MongoDB", "FastAPI", "Kubernetes"],
        "tertiary": ["Elasticsearch", "SOAP", "Redis", "ClickHouse", "Django"]
    },
    "Бизнес-аналитик": {
        "primary": ["SQL", "BPMN", "BI", "Power BI", "UML"],
        "secondary": ["Jira", "Confluence", "Бизнес-процесс", "SWOT-анализ", "Agile"],
        "tertiary": ["DFD", "Flowchart", "ERD", "ИDEF", "Business studio"]
    },
    "Разработчик 3D-игр на языке JS": {
        "primary": ["Babylon JS", "Оформление кода в объекты и классы", "Знание 3D движков", "Основы линейной алгебры", "ООП", "JavaScript", "TypeScript", "HTML", "CSS"],
        "secondary": ["Создание интерактивных 3D-сцен", "Оптимизация производительности игр", "Интеграция WebGL", "Работа с текстурами и шейдерами", "Обработка пользовательского ввода"],
        "tertiary": ["Работа с физическими движками", "Проектирование игровых уровней", "Коллаборация с дизайнерами и художниками", "Деплой веб-приложений", "Тестирование и отладка 3D-игр"]
    }
}


In [452]:
class CandidateRecommender:
    def __init__(self, github_token, enable_git_parsing=True):
        # Инициализация модели и параметров
        self.tfidf_vectorizer = TfidfVectorizer()
        self.github_token = github_token
        self.enable_git_parsing = enable_git_parsing  # Управление парсингом GitHub

    # Парсинг GitHub, если включен флаг enable_git_parsing
    def parse_github(self, df):
        if not self.enable_git_parsing:
            print("Используются существующие данные.")
            return df

        def extract_github_username(url):
            if pd.isna(url) or 'github.com' not in url:
                return None
            return url.split('github.com/')[-1].split('/')[0]

        df['GitHub_Username'] = df['GitHub'].apply(extract_github_username)

        for index, row in df.iterrows():
            username = row['GitHub_Username']
            if username:
                print(f"Получение данных для пользователя: {username}")
                github_data = self.__get_github_data(username)
                if github_data:
                    for key, value in github_data.items():
                        df.at[index, key] = value
                time.sleep(1)

        return df

    # Приватный метод получения данных профиля пользователя GitHub
    def __get_github_data(self, username):
        headers = {'Authorization': f'token {self.github_token}'}
        base_url = f'https://api.github.com/users/{username}'
        try:
            profile_response = requests.get(base_url, headers=headers)
            if profile_response.status_code == 401:
                print("Ошибка 401: Неверный или истёкший токен GitHub.")
                return None
            elif profile_response.status_code == 404:
                print(f"Пользователь {username} не найден.")
                return None
            elif profile_response.status_code != 200:
                print(f"Ошибка {profile_response.status_code} при запросе профиля {username}")
                return None

            repos_url = f'{base_url}/repos'
            repos_response = requests.get(repos_url, headers=headers)
            if repos_response.status_code != 200:
                print(f"Ошибка {repos_response.status_code} при запросе репозиториев {username}")
                return None

            repos_data = repos_response.json()
            languages = set()
            for repo in repos_data:
                languages_url = repo.get('languages_url')
                if languages_url:
                    languages_response = requests.get(languages_url, headers=headers)
                    if languages_response.status_code == 200:
                        repo_languages = languages_response.json()
                        languages.update(repo_languages.keys())

            return {
                'GitHub_Languages': ', '.join(languages)
            }
        except requests.RequestException as e:
            print(f"Ошибка запроса для {username}: {e}")
            return None

    # Переименование колонок для унификации
    def rename_columns(self, df):   
        df = df.drop('Навыки (из справочника)', axis=1)
        return df.rename(columns={
            'ID': 'ID',
            'О себе': 'About',
            'Портфолио': 'Portfolio',
            'Навыки': 'Skills',
            'Специальность->Название': 'Specialization',
            'GitHub_Languages': 'GitHub_Tech_Stack'
        })

    # Подготовка данных
    def prepare_data(self, csv_file_path, excel_file_path):
        # Чтение данных из CSV и Excel
        df_csv = pd.read_csv(csv_file_path)
        df_excel = pd.read_excel(excel_file_path)

        #парсинг гита

        # Приведение идентификаторов к строковому формату для объединения
        df_csv['User_ID'] = df_csv['User_ID'].fillna(0).astype(int).astype(str).str.strip()
        df_excel['ID'] = df_excel['ID'].astype(str).str.strip()
        
        # Объединение данных по столбцам 'User_ID' и 'ID'
        df = pd.merge(
            # df_csv[['User_ID', 'name', 'interpretation']],
            df_csv[['User_ID', 'interpretation']],
            df_excel,
            left_on='User_ID',
            right_on='ID', 
            how='outer'
        )

        df = self.parse_github(df)
        
        df = self.rename_columns(df)        
        # Очистка текста в объединенных данных
        def clean_text(text):
            if not text or not isinstance(text, str):
                return ''
            words = text.split()
            return ' '.join(word for word in words if word not in ['нет', 'информации', 'не', 'указано'])

        # df = df.drop('ID', axis=1)
        
        if 'GitHub_Tech_Stack' not in df.columns:
            df['GitHub_Tech_Stack'] = ''

        df['Combined_Text'] = (
            df['Specialization'].fillna('').astype(str).apply(lambda x: '' if x.strip().lower() in ['не указано', 'нет информации'] else x) + ' ' +
            df['Skills'].fillna('').astype(str).apply(lambda x: '' if x.strip().lower() in ['не указано', 'нет информации'] else x) + ' ' +
            df['About'].fillna('').astype(str).apply(lambda x: '' if x.strip().lower() in ['не указано', 'нет информации'] else x) + ' ' +
            df['Portfolio'].fillna('').astype(str).apply(lambda x: '' if x.strip().lower() in ['не указано', 'нет информации'] else x) + ' ' +
            df['GitHub_Tech_Stack'].fillna('').astype(str).apply(lambda x: '' if x.strip().lower() in ['не указано', 'нет информации'] else x)
        ).str.lower()

        df['Combined_Text'] = df['Combined_Text'].apply(clean_text)

        return df
    
    # Генерация текстов для специальностей с кастомными весами
    def generate_specialization_texts(self, specializations):
        texts = []
        for spec, skills in specializations.items():
            spec_name = ' '.join([spec.lower()] * 10)
            primary_skills = ' '.join([skill.lower() for skill in skills["primary"]] * 5)
            secondary_skills = ' '.join([skill.lower() for skill in skills["secondary"]] * 3)
            tertiary_skills = ' '.join([skill.lower() for skill in skills["tertiary"]])
            texts.append(f"{spec_name} {primary_skills} {secondary_skills} {tertiary_skills}")
        return texts


    def add_belbin_roles(self, df):
        def parse_belbin_roles(interpretation):
            try:
                roles = json.loads(interpretation) if isinstance(interpretation, str) else []
                return [{"name": role["name"], "score": role["score"]} for role in roles]
            except Exception as e:
                print(f"Parsing error: {e}, data: {interpretation}")
                return []

        def calculate_top_roles_with_percentages(roles):
            if not roles or not isinstance(roles, list):
                return {}
            
            sorted_roles = sorted(roles, key=lambda x: x["score"], reverse=True)[:3]
            total_score = sum(role["score"] for role in sorted_roles)
            if total_score == 0:
                return {role["name"]: 0 for role in roles}
            
            top_roles_percentages = {
                role["name"]: round((role["score"] / total_score) * 100, 2)
                for role in sorted_roles
            }
            
            all_roles = [
                "Координатор", "Генератор идей", "Оценщик", "Коллективист",
                "Доводчик", "Реализатор", "Формирователь", "Специалист", "Разведчик"
            ]
            return {role: top_roles_percentages.get(role, 0) for role in all_roles}

        role_columns = [
            "Координатор", "Генератор идей", "Оценщик", "Коллективист",
            "Доводчик", "Реализатор", "Формирователь", "Специалист", "Разведчик"
        ]

        df['parsed_roles'] = df['interpretation'].apply(parse_belbin_roles)

        for role in role_columns:
            df[role] = df['parsed_roles'].apply(
                lambda roles: calculate_top_roles_with_percentages(roles).get(role, 0)
            )

        df.drop(columns=['parsed_roles'], inplace=True)
        return df

    # Поиск кандидатов
    def find_candidates(self, candidates_df, specializations, input_data=None):
        # Нормализуем справочник специализаций
        
        def normalize_specializations(specializations):
            normalized = {}
            for spec, skills in specializations.items():
                normalized[spec.lower()] = {
                    "primary": [skill.lower() for skill in skills["primary"]],
                    "secondary": [skill.lower() for skill in skills["secondary"]],
                    "tertiary": [skill.lower() for skill in skills["tertiary"]],
                }
            return normalized
        
        specializations = normalize_specializations(specializations)

        if input_data is None:
            specializations_list = list(specializations.keys())
            specialization_texts = self.generate_specialization_texts(specializations)

            # Нормализуем текст кандидатов
            combined_texts = candidates_df['Combined_Text'].fillna('').str.lower().tolist()
            all_texts = specialization_texts + combined_texts

            # Вычисляем TF-IDF матрицу
            tfidf_matrix = self.tfidf_vectorizer.fit_transform(all_texts)
            specialization_matrix = tfidf_matrix[:len(specializations_list)]
            candidate_matrix = tfidf_matrix[len(specializations_list):]

            similarity_matrix = cosine_similarity(candidate_matrix, specialization_matrix)

            matched_specializations = []
            for candidate_index, similarities in enumerate(similarity_matrix):
                # Усиливаем совпадение по названию специальности
                for idx, spec in enumerate(specializations_list):
                    if spec in combined_texts[candidate_index]:  # Сравнение уже нормализовано
                        similarities[idx] += 1.5

                # Усиливаем совпадение по навыкам
                for idx, spec in enumerate(specializations_list):
                    skills = specializations[spec]
                    primary_matches = sum(skill in combined_texts[candidate_index] for skill in skills["primary"])
                    secondary_matches = sum(skill in combined_texts[candidate_index] for skill in skills["secondary"])
                    tertiary_matches = sum(skill in combined_texts[candidate_index] for skill in skills["tertiary"])

                    similarities[idx] += primary_matches * 2.0  # Усиление веса первичных навыков
                    similarities[idx] += secondary_matches * 1.0
                    similarities[idx] += tertiary_matches * 0.5

                # Сортируем совпадения по убыванию
                sorted_matches = sorted(
                    zip(specializations_list, similarities),
                    key=lambda x: x[1],
                    reverse=True
                )

                # Ограничиваемся топ-3 совпадениями с порогом > 0.2
                filtered_matches = [
                    f"{spec} ({round(score, 4)})" for spec, score in sorted_matches if score > 0.2
                ][:3]

                # Если ничего не найдено, добавляем "Нет подходящей специальности"
                if not filtered_matches:
                    filtered_matches = ["Нет подходящей специальности"]

                matched_specializations.append(', '.join(filtered_matches))

            candidates_df['Matched_Specializations'] = matched_specializations
            return candidates_df

        elif isinstance(input_data, list):
            # Обработка поиска по навыкам
            combined_texts = candidates_df['Combined_Text'].fillna('').tolist()
            all_texts = [' '.join(input_data)] + combined_texts

            tfidf_matrix = self.tfidf_vectorizer.fit_transform(all_texts)
            user_vector = tfidf_matrix[0]
            candidate_matrix = tfidf_matrix[1:]

            # Вычисление косинусного сходства
            similarities = cosine_similarity(user_vector, candidate_matrix).flatten()
            candidates_df['Similarity'] = similarities
            return candidates_df.sort_values(by='Similarity', ascending=False)

        else:
            raise ValueError("Входными данными может быть список навыков либо название специализации.")

    # Получение топ-N кандидатов
    def find_top_candidates_by_specialization_from_results(self, candidates_df, specializations_str, top_n=5):

        if 'Matched_Specializations' not in candidates_df.columns:
            raise ValueError("Колонка 'Matched_Specializations' отсутствует в DataFrame. Убедитесь, что find_candidates выполнен.")

        required_specializations = [spec.strip().lower() for spec in specializations_str.split(',')]

        def calculate_relevance(specializations_str):
            if not isinstance(specializations_str, str):
                return 0, 0 
            
            matches = [match.strip() for match in specializations_str.split(',')]
            match_count = 0
            max_similarity = 0
            
            for match in matches:
                for specialization in required_specializations:
                    if specialization in match:
                        try:
                            similarity = float(match.split('(')[-1].strip(')'))
                            max_similarity = max(max_similarity, similarity)
                            match_count += 1
                        except ValueError:
                            continue
            return match_count, max_similarity

        
        candidates_df['Match_Count'], candidates_df['Max_Similarity'] = zip(
            *candidates_df['Matched_Specializations'].apply(calculate_relevance)
        )

        candidates_df = candidates_df.sort_values(
            by=['Match_Count', 'Max_Similarity'], 
            ascending=[False, False]
        )

        return candidates_df.head(top_n).drop(columns=['Match_Count', 'Max_Similarity'], errors='ignore')
    
    # Формирование команды под указанный тип заказа / проекта)
    def build_team_for_project(self, candidates_df, order_type, team_size=5):
       
        if order_type not in order_types:
            raise ValueError(f"Тип заказа '{order_type}' отсутствует в справочнике.")

        required_specializations = order_types[order_type]
        team = []
        remaining_candidates = candidates_df.copy()

        # Подбор кандидатов для каждой специализации
        for specialization in required_specializations:
            if len(team) >= team_size:
                break

            # Находим кандидатов для текущей специализации
            candidates_for_specialization = self.find_top_candidates_by_specialization_from_results(
                remaining_candidates,
                specialization,
                top_n=1  
            )

            if not candidates_for_specialization.empty:
                selected_candidate = candidates_for_specialization.iloc[0].to_dict()  
                team.append(selected_candidate)
                remaining_candidates = remaining_candidates[remaining_candidates['ID'] != selected_candidate['ID']]
        # В случае, если указано количество человек больше, чем специализаций в заказе, то 
        # добираем менее релевантных кандидатов
        if len(team) < team_size:
            # Вычисляем среднюю релевантность кандидатов к задачам заказа
            def calculate_relevance(row):
                total_relevance = 0
                for specialization in required_specializations:
                    relevance = self.calculate_specialization_relevance(row['Matched_Specializations'], specialization)
                    total_relevance += relevance
                return total_relevance

            remaining_candidates['Relevance_To_Order'] = remaining_candidates.apply(calculate_relevance, axis=1)

            # Сортируем оставшихся кандидатов по релевантности
            additional_candidates = remaining_candidates.sort_values(by='Relevance_To_Order', ascending=False)
            
            # Добираем недостающих кандидатов
            additional_candidates = additional_candidates.head(team_size - len(team)).to_dict('records')
            team.extend(additional_candidates)

        return pd.DataFrame(team)

    # Функция для вычисления релевантности кандидата к специализации.
    def calculate_specialization_relevance(self, matched_specializations, specialization):
        if not isinstance(matched_specializations, str):
            return 0
        
        matches = [match.strip() for match in matched_specializations.split(',')]
        for match in matches:
            if specialization.lower() in match.lower():
                try:
                    return float(match.split('(')[-1].strip(')'))
                except ValueError:
                    return 0
        return 0
    

    # Подбор кандидатов по специализациям и ролям Белбина с сортировкой по релевантности.
    def build_team_by_specialization_and_belbin(self, candidates_df, current_team, order_type, max_team_size=20):
        if not isinstance(current_team, pd.DataFrame):
            raise ValueError("current_team должен быть DataFrame.")

        candidates_df['User_ID'] = candidates_df['User_ID'].astype(str).str.strip()
        current_team['User_ID'] = current_team['User_ID'].astype(str).str.strip()

        current_ids = current_team['User_ID'].tolist()
        remaining_candidates = candidates_df[~candidates_df['User_ID'].isin(current_ids)].copy()

        specialization_team = self.build_team_for_project(remaining_candidates, order_type, team_size=max_team_size)

        extended_team = pd.concat([current_team, specialization_team], ignore_index=True)

        extended_team = extended_team.drop_duplicates(subset='User_ID', keep='first')

        remaining_candidates = candidates_df[~candidates_df['User_ID'].isin(current_ids)].copy()

        balanced_team = self.balance_team_by_belbin(extended_team, remaining_candidates, max_team_size, current_ids)

        return balanced_team

    # Балансировка команды по тесту Белбина
    def balance_team_by_belbin(self, extended_team, remaining_candidates, max_team_size, current_team_ids):
        role_columns = [
            "Координатор", "Генератор идей", "Оценщик", "Коллективист",
            "Доводчик", "Реализатор", "Формирователь", "Специалист", "Разведчик"
        ]
        current_roles = extended_team[role_columns].sum()

       
        avg_role_count = max_team_size / len(role_columns)
        missing_roles = {role: max(0, int(avg_role_count - current_roles[role])) for role in role_columns}

        for role, count in missing_roles.items():
            if count > 0:
                role_candidates = remaining_candidates[remaining_candidates[role] > 0].copy()
                role_candidates = role_candidates.sort_values(by=role, ascending=False)

                for _ in range(count):
                    if role_candidates.empty:
                        break
                    selected_candidate = role_candidates.iloc[0]
                    extended_team = pd.concat(
                        [extended_team, pd.DataFrame([selected_candidate.to_dict()])], ignore_index=True
                    )
                    remaining_candidates = remaining_candidates[
                        remaining_candidates['User_ID'] != selected_candidate['User_ID']
                    ]

        extended_team = extended_team[~extended_team['User_ID'].isin(current_team_ids)]

        return extended_team.head(max_team_size)


In [453]:
order_types = {
    "Веб-сервис": [
        "Фронтенд-разработчик", 
        "Бэкенд-разработчик", 
        "Фулстек-разработчик", 
        "Дизайнер", 
        "DevOps инженер"
    ],
    "Аналитическая система": [
        "Аналитик данных", 
        "Специалист по Data Science", 
        "Инженер данных", 
        "Системный аналитик"
    ],
    "Разработка 3D-игры": [
        "Разработчик 3D-игр на языке JS", 
        "Дизайнер", 
        "Фулстек-разработчик", 
        "Тестировщик"
    ],
    "Стартап по Data Science": [
        "Специалист по Data Science", 
        "Инженер данных", 
        "DevOps инженер", 
        "Бизнес-аналитик", 
        "Менеджер"
    ],
    "Создание корпоративного портала": [
        "Системный аналитик", 
        "Бизнес-аналитик", 
        "Фулстек-разработчик", 
        "Тестировщик", 
        "Дизайнер"
    ],
}

## Пример работы

### Создание экземпляра класса без парсинга

In [1]:
github_token = 'your tocken'

recommender = CandidateRecommender(github_token=github_token, enable_git_parsing=False)

NameError: name 'CandidateRecommender' is not defined

In [462]:
# csv_file_path = "./test_results_03.11_ID.csv"
csv_file_path = "./test_generation2.csv"
excel_file_path = "./analys.xlsx"

In [463]:
candidates_df = recommender.prepare_data(csv_file_path, excel_file_path)
candidates_df = recommender.add_belbin_roles(candidates_df)

Используются существующие данные.


### Присвоение кандидатам специальности

In [464]:
result = recommender.find_candidates(candidates_df, specializations)

In [465]:
result[['ID', 'Combined_Text', 'Matched_Specializations', 'Координатор', 'Генератор идей', 'Оценщик', 'Коллективист', 'Доводчик', 'Реализатор', 'Формирователь', 'Специалист', 'Разведчик']].sample(10)

,ID,Combined_Text,Matched_Specializations,Координатор,Генератор идей,Оценщик,Коллективист,Доводчик,Реализатор,Формирователь,Специалист,Разведчик
167,9167,,Нет подходящей специальности,0.00,40.00,0.0,0.00,11.43,0.00,0.00,0.00,48.57
719,9719,,Нет подходящей специальности,14.29,0.00,0.0,0.00,44.64,0.00,0.00,41.07,0.00
598,9598,,Нет подходящей специальности,0.00,0.00,0.0,0.00,0.00,33.96,0.00,33.96,32.08
838,NaN,,Нет подходящей специальности,0.00,0.00,0.0,0.00,31.58,0.00,10.53,0.00,57.89
390,9390,"менеджер докторант, специалист по ии, пожарная безопасность вмф, ниокр по гоз",менеджер (1.5847),47.54,0.00,0.0,18.03,0.00,0.00,34.43,0.00,0.00
205,9205,,Нет подходящей специальности,0.00,35.94,0.0,0.00,20.31,0.00,0.00,43.75,0.00
290,9290,"фулстек-разработчик swagger, git, postman, javascript, html, css, vue.js, node.js, vuex, docker, angular, scss, typescript, react redux, react, django, ci/cd, rxjs, postgresql мой основной стек прописан в резюме. помимо него ещё django + asp net. беком занимаюсь самостоятельно и основном для запуска своих проектов на сервере 185.173.94.160 + docker, занимаюсь развитием в сфере devops и backend и ищу проекты для получения навыков сайт для рсо (российские студенческие отряды)","фронтенд-разработчик (13.2531), фулстек-разработчик (12.7636), разработчик 3d-игр на языке js (8.0915)",32.65,0.00,0.0,0.00,0.00,40.82,0.00,0.00,26.53
26,9026,,Нет подходящей специальности,0.00,0.00,0.0,0.00,37.50,0.00,0.00,34.38,28.12
408,9408,"agile, техническая документация, excel, бпла, 3d-печать, python, sql, навыки деловой коммуникации","аналитик данных (6.124), инженер данных (4.0966), фулстек-разработчик (4.0824)",0.00,17.19,0.0,29.69,53.12,0.00,0.00,0.00,0.00
77,9077,аналитик данных,"аналитик данных (2.1629), системный аналитик (0.2881), инженер данных (0.2532)",14.04,28.07,0.0,57.89,0.00,0.00,0.00,0.00,0.00


### Сбор команды по специальностям и Белбину с учетом уже выбранного / выбранных специалистов

Определяем текущую команду (должен быть хотя бы 1 человек, чтобы рекомендация отталкивалась от него)

In [ ]:
current_team = pd.DataFrame([
    candidates_df[candidates_df['User_ID'] == '9015'].iloc[0],
    # candidates_df[candidates_df['User_ID'] == '9018'].iloc[0],
    # candidates_df[candidates_df['User_ID'] == '9742'].iloc[0]
])

In [468]:
current_team[['User_ID', 'interpretation', 'Matched_Specializations']]

,User_ID,interpretation,Matched_Specializations
15,9015,"[{""name"":""Специалист"",""score"":18},{""name"":""Доводчик"",""score"":17},{""name"":""Реализатор"",""score"":14},{""name"":""Генератор идей"",""score"":6},{""name"":""Оценщик"",""score"":5},{""name"":""Формирователь"",""score"":5},{""name"":""Коллективист"",""score"":3},{""name"":""Координатор"",""score"":2},{""name"":""Разведчик"",""score"":0}]","фулстек-разработчик (16.7387), фронтенд-разработчик (15.7364), бэкенд-разработчик (11.1207)"


In [469]:
final_team = recommender.build_team_by_specialization_and_belbin(
    candidates_df=candidates_df,
    current_team=current_team,
    order_type="Веб-сервис",
    max_team_size=15
)
final_team[["User_ID", "interpretation", "Matched_Specializations"]]

,User_ID,interpretation,Matched_Specializations
1,9027,"[{""name"":""Доводчик"",""score"":12},{""name"":""Реализатор"",""score"":11},{""name"":""Специалист"",""score"":11},{""name"":""Генератор идей"",""score"":10},{""name"":""Коллективист"",""score"":8},{""name"":""Разведчик"",""score"":6},{""name"":""Координатор"",""score"":5},{""name"":""Формирователь"",""score"":4},{""name"":""Оценщик"",""score"":3}]","фронтенд-разработчик (15.2101), разработчик 3d-игр на языке js (8.0746), фулстек-разработчик (5.0809)"
2,9018,"[{""name"":""Специалист"",""score"":13},{""name"":""Коллективист"",""score"":11},{""name"":""Координатор"",""score"":9},{""name"":""Формирователь"",""score"":9},{""name"":""Разведчик"",""score"":8},{""name"":""Доводчик"",""score"":6},{""name"":""Реализатор"",""score"":5},{""name"":""Оценщик"",""score"":5},{""name"":""Генератор идей"",""score"":4}]","бэкенд-разработчик (11.7593), фулстек-разработчик (10.6817), devops инженер (7.6503)"
3,9210,"[{""name"":""Оценщик"",""score"":25},{""name"":""Коллективист"",""score"":14},{""name"":""Доводчик"",""score"":13},{""name"":""Формирователь"",""score"":6},{""name"":""Разведчик"",""score"":4},{""name"":""Генератор идей"",""score"":4},{""name"":""Реализатор"",""score"":2},{""name"":""Координатор"",""score"":1},{""name"":""Специалист"",""score"":1}]","фулстек-разработчик (15.5651), фронтенд-разработчик (10.4091), бэкенд-разработчик (7.1769)"
4,9742,"[{""name"":""Реализатор"",""score"":28},{""name"":""Оценщик"",""score"":22},{""name"":""Формирователь"",""score"":7},{""name"":""Доводчик"",""score"":6},{""name"":""Специалист"",""score"":5},{""name"":""Координатор"",""score"":2},{""name"":""Генератор идей"",""score"":0},{""name"":""Коллективист"",""score"":0},{""name"":""Разведчик"",""score"":0}]","дизайнер (15.1496), маркетолог (2.0387)"
5,9089,"[{""name"":""Разведчик"",""score"":26},{""name"":""Реализатор"",""score"":18},{""name"":""Генератор идей"",""score"":10},{""name"":""Доводчик"",""score"":8},{""name"":""Формирователь"",""score"":5},{""name"":""Специалист"",""score"":2},{""name"":""Коллективист"",""score"":1},{""name"":""Оценщик"",""score"":0},{""name"":""Координатор"",""score"":0}]","devops инженер (15.1326), фулстек-разработчик (9.6678), бэкенд-разработчик (9.1882)"
6,9314,"[{""name"":""Генератор идей"",""score"":27},{""name"":""Оценщик"",""score"":16},{""name"":""Реализатор"",""score"":10},{""name"":""Координатор"",""score"":7},{""name"":""Разведчик"",""score"":4},{""name"":""Специалист"",""score"":3},{""name"":""Формирователь"",""score"":2},{""name"":""Доводчик"",""score"":1},{""name"":""Коллективист"",""score"":0}]","фронтенд-разработчик (12.245), фулстек-разработчик (10.7409), дизайнер (8.1472)"
7,9290,"[{""name"":""Реализатор"",""score"":20},{""name"":""Координатор"",""score"":16},{""name"":""Разведчик"",""score"":13},{""name"":""Формирователь"",""score"":11},{""name"":""Специалист"",""score"":6},{""name"":""Коллективист"",""score"":2},{""name"":""Оценщик"",""score"":2},{""name"":""Доводчик"",""score"":0},{""name"":""Генератор идей"",""score"":0}]","фронтенд-разработчик (13.2531), фулстек-разработчик (12.7636), разработчик 3d-игр на языке js (8.0915)"
8,9049,"[{""name"":""Формирователь"",""score"":31},{""name"":""Специалист"",""score"":26},{""name"":""Оценщик"",""score"":10},{""name"":""Разведчик"",""score"":3},{""name"":""Реализатор"",""score"":0},{""name"":""Генератор идей"",""score"":0},{""name"":""Координатор"",""score"":0},{""name"":""Доводчик"",""score"":0},{""name"":""Коллективист"",""score"":0}]","бэкенд-разработчик (9.6263), фулстек-разработчик (8.5883), devops инженер (6.7149)"
9,9460,"[{""name"":""Доводчик"",""score"":34},{""name"":""Реализатор"",""score"":31},{""name"":""Разведчик"",""score"":5},{""name"":""Оценщик"",""score"":0},{""name"":""Генератор идей"",""score"":0},{""name"":""Формирователь"",""score"":0},{""name"":""Специалист"",""score"":0},{""name"":""Коллективист"",""score"":0},{""name"":""Координатор"",""score"":0}]","фронтенд-разработчик (14.928

### Подбор кандидатов под определенный тип заказа

In [470]:
order_types = {
    "Веб-сервис": [
        "Фронтенд-разработчик", 
        "Бэкенд-разработчик", 
        "Фулстек-разработчик", 
        "Дизайнер", 
        "DevOps инженер"
    ],
    "Аналитическая система": [
        "Аналитик данных", 
        "Специалист по Data Science", 
        "Инженер данных", 
        "Системный аналитик"
    ],
    "Разработка 3D-игры": [
        "Разработчик 3D-игр на языке JS", 
        "Дизайнер", 
        "Фулстек-разработчик", 
        "Тестировщик"
    ],
    "Стартап по Data Science": [
        "Специалист по Data Science", 
        "Инженер данных", 
        "DevOps инженер", 
        "Бизнес-аналитик", 
        "Менеджер"
    ],
    "Создание корпоративного портала": [
        "Системный аналитик", 
        "Бизнес-аналитик", 
        "Фулстек-разработчик", 
        "Тестировщик", 
        "Дизайнер"
    ],
}

In [471]:
order_type = "Веб-сервис"
team = recommender.build_team_for_project(candidates_df, order_type, team_size=20)
print("Состав команды для заказа:")
team[['ID', 'Matched_Specializations']]

Состав команды для заказа:


,ID,Matched_Specializations
0,9015,"фулстек-разработчик (16.7387), фронтенд-разработчик (15.7364), бэкенд-разработчик (11.1207)"
1,9018,"бэкенд-разработчик (11.7593), фулстек-разработчик (10.6817), devops инженер (7.6503)"
2,9210,"фулстек-разработчик (15.5651), фронтенд-разработчик (10.4091), бэкенд-разработчик (7.1769)"
3,9742,"дизайнер (15.1496), маркетолог (2.0387)"
4,9089,"devops инженер (15.1326), фулстек-разработчик (9.6678), бэкенд-разработчик (9.1882)"
5,9314,"фронтенд-разработчик (12.245), фулстек-разработчик (10.7409), дизайнер (8.1472)"
6,9290,"фронтенд-разработчик (13.2531), фулстек-разработчик (12.7636), разработчик 3d-игр на языке js (8.0915)"
7,9049,"бэкенд-разработчик (9.6263), фулстек-разработчик (8.5883), devops инженер (6.7149)"
8,9460,"фронтенд-разработчик (14.9289), фулстек-разработчик (9.7904), тестировщик (6.5706)"
9,9447,"фулстек-разработчик (9.2918), бэкенд-разработчик (7.8135), фронтенд-разработчик (7.2476)"


### Вывод топ-5 подходящих кандидатов по специальности

In [472]:
input_specialization = "Дизайнер, Фронтенд-разработчик, менеджер"
top_n = 10

top_candidates = recommender.find_top_candidates_by_specialization_from_results(
    result, 
    specializations_str=input_specialization, 
    top_n=top_n
)

print(f"Топ-{top_n} кандидатов по специальности '{input_specialization}':")
top_candidates[['ID', 'Combined_Text', 'Matched_Specializations']]

Топ-10 кандидатов по специальности 'Дизайнер, Фронтенд-разработчик, менеджер':


,ID,Combined_Text,Matched_Specializations
314,9314,"фулстек-разработчик git, json, mysql, postman, api, адаптивная верстка, https, react, nestjs, node.js, vue.js, css, bootstrap, база данных, javascript, scss, sass, adobe photoshop, ux/ui, figma закончил программу бакалавра в урфу по специальности ""прикладная информатика"". выпускник курса 1т ""web-разработчик 2024"" для дипломной работы написал front-часть приложения на reactjs+typescript в команде с другими студентами, проект был собран с помощью docker. имею навыки в растровой и векторной графике, регулярно пользуюсь ими для различных целей учебный проект на quasar с firebase","фронтенд-разработчик (12.245), фулстек-разработчик (10.7409), дизайнер (8.1472)"
765,9765,"дизайнер умение работать в команде, scrum, miro, google docs, excel, подготовка презентации, cjm, ux/ui, ux, figma, ui, web design, sketch, landing page я ux/ui дизайнер, люблю работать в команде, придумывать идеи для пользовательского интерфейса. обладаю насмотренностью. умею работать с пользовательским сценарием. стараюсь создавать удобные и при этом эстетичные интерфейсы. готова развиваться дальше.","дизайнер (10.8013), менеджер (2.5466), маркетолог (2.0435)"
19,9019,"менеджер управление проектом, jira, excel, проведение презентации, ms project, тайм-менеджмент, miro, google docs, confluence, html, ux/ui, ориентация на результат, проектная документация, документооборот, бизнес-процесс, trello закончила курс ux ui дизайнера на 1т спринт, была менеджером и дизайнером в учебном проекте, вместе с разработчикам с курса сделали сайт для оконной компании, в 2023 сделала команду пиксель панкс, сделали макет для платформы 1т мир ссылка на тильду http://pixelpunks.tilda.ws.tilda.ws/page36413567.html ссылка на беханс https://www.behance.net/nastya01090ae4","менеджер (10.2166), дизайнер (7.0974), бизнес-аналитик (3.0783)"
294,9294,"дизайнер подготовка презентации, ux/ui, инфографика, figma, adobe photoshop, 1с: предприятие, 1с, html, vue.js, css, техническая поддержка закончил бакалавриат по направлению фундаментальной информатики и информационных технологий. работаю в it компании 1с разработчиком. наша команда занимается разработкой и поддержкой медицинской информационной системы, используемой в клиниках по всей россии. мечтаю попробовать себя в сфере дизайна, веба или ui/ux. считаю себя новичком в этой сфере, но имею пару учебных проектов. активный пользователь figma, photoshop, powerpoint. имею опыт в разработке красивых и информативных презентаций. мобильное приложение для врача на платформе 1с предприятие, разработка макета мобильного приложения медицинского агрегатора, презентация проекта медицинского агрегатора, презентация проекта green taxi, презентация на тему урбанистики, презентация на тему kulturkampf, различные проекты учебных презентаций, буклет-реклама туристического направления","дизайнер (9.6561), фронтенд-разработчик (5.0413), специалист по 1с (4.7952)"
339,9339,"фронтенд-разработчик javascript, devtools, html, css, php, seo, яндекс директ, адаптивная верстка, bootstrap, jquery, ftp, adobe photoshop, adobe illustrator, wordpress, tilda начинающий специалист, активно прохожу обучение и развиваюсь. знаниe html, css, bootstrap, git, jquery,js.","фронтенд-разработчик (8.815), разработчик 3d-игр на языке js (6.0996), дизайнер (5.2744)"
490,9490,"дизайнер miro, figma, landing page, web design, tilda, adobe photoshop, html, css всем привет, я гараев эльдар, начинающий веб дизайнер, мне нравится делать интересный дизайн и работать с нестандартными проектами, которые в дальнейшем могут облегчить жизнь многим людям. я имею опыт работы в figma, tilda, adobe photoshop, а так же html и css, так что я могу сделать хороший дизайн сайта, а так же написать его код. так же я имею небольшой опыт в рекламе и аналитических данных для разработки сайта. parla inform","дизайнер (8.7979), фронтенд-разработчик (4.0611), разработчик 3d-игр на языке js (4.0246)"
356,9356,"дизайнер adobe photoshop, подготовка презентации, графич

## Поиск по навыкам

In [473]:
input_skills = ["Figma", "Html", "JAVASCRIPT", 'Python']
top_n = 5

top_candidates_by_skills = recommender.find_candidates(candidates_df, specializations, input_data=input_skills)

print(f"Топ-{top_n} кандидатов по навыкам {', '.join(input_skills)}:")
top_candidates_by_skills[['ID', 'Combined_Text', 'Similarity']].head(top_n)

Топ-5 кандидатов по навыкам Figma, Html, JAVASCRIPT, Python:


,ID,Combined_Text,Similarity
17,9017,"python, pandas, sql, mysql, git, c/c++, c#, html, css, javascript",0.493588
178,9178,"figma, web design, adobe photoshop, ux/ui, jira, python, mysql, pandas, numpy, matplotlib, scikit-learn, css, html",0.313785
37,9037,"sql, python, postgresql, pandas, matplotlib, git, javascript, vue.js",0.302480
447,9447,"бэкенд-разработчик python, sql, git, javascript, html, css, c/c++, linux, ubuntu, debian, shell scripting, bash, node.js, docker, tcp/ip",0.243667
89,9089,"devops инженер vmware, git, sql, python, tcp/ip, ci/cd, mysql, linux, docker, nginx, kubernetes, ansible, prometheus, grafana, postgresql, pyspark, pandas, numpy, figma, javascript",0.234576
